In [10]:
import numpy as np
from numpy import array
from numpy.linalg import norm, det
from numpy.linalg import solve as solve_out_of_the_box
from numpy.random import uniform
from numpy import vectorize
from operator import add


def multiply(x, a):
    return x * a

def vect_mult():
    return vectorize(multiply)

def gauss(a, b):
    ab = np.concatenate((a, np.array([b]).T), axis=1) # concatenate заодно и скопирует 
    d = len(ab)  # размер по старшему измерению
    for i in range(d):
        j = i
        while j < len(a[i]) and ab[i][j] == 0:
            j += 1
        if j == len(a[i]):
            continue
        r = vectorize(multiply)
        ab[i] = r (ab[i], 1/ab[i][j])
        for k in range(i+1, d):
            ab[k] = ab[k] - ab[k][j]*ab[i]
       
    #проверка на совместимость 
    for i in range(d):            
        nonzero_r = False
        for j in range(len(a[i])):
            if abs(ab[i][j]) > 10**(-10):
                nonzero_r = True
        if nonzero_r == False and abs(ab[i][-1]) > 10**(-10):
            return 'решений нет'
    
    solution = [0]*len(ab)
    for i in range(len(ab)-1, -1, -1):
        res = ab[i][-1]
        for j in range(len(ab[i])-2, i-1, -1):
            res -= ab[i][j]*solution[j]
        if ab[i][i] != 0:
            solution[i] = res / ab[i][i]
    return solution

N=100
SMALL = 1e-5

def test_error(solver_function):
    # Сгенерируем хорошо обусловленную систему
    while True:
        a = uniform(0.0, 1.0, (N, N))
        b = uniform(0.0, 1.0, (N,  ))

        d = det(a)
        if abs(d) <= SMALL:  # Определитель маленький — плохо
            # print(f"det: {d}")
            continue  # Попробуем ещё
        if d < 0.0:  # Отрицательный — поменяем знак
            # print(f"det: {d}")
            a = -a

        try:
            oob_solution = solve_out_of_the_box(a, b)
        except Exception as e:  # Всё-таки система плохая
            # print(f"exc: {e}")
            continue  # Попробуем ещё

        sb = a @ oob_solution
        if norm(sb - b, ord=1) > SMALL:
            # print("Bad solution...")
            continue  # Всё же не очень система получилась =)
        
        break # Всё, считаем, что мы случайно сгенерировали хорошую систему

    tested_solution = solver_function(a, b)
    return norm(tested_solution - oob_solution, ord=1)

test_error(gauss)
 

2.524071150656823e-10